In [ ]:
# You'll start seeing this cell in most lectures.
# It exists to hide all of the import statements and other setup
# code we need in lecture notebooks.
from dsc80_utils import *

# Lecture 2 – DataFrame Fundamentals

## DSC 80, Winter 2024

There was a <b><a href="https://dsc80.com/resources/lectures/lec02/pre-lec02.html"><span style="color:#7b40c7">Pre-Lecture Reading</span></a></span></b> for this lecture – we'll assume you've done it.


### Announcements 📣

- **Today's lecture is on [Zoom](https://ucsd.zoom.us/my/rampure)**, but we'll be back in-person next week.
- Lab 1 is released, and is due **Wednesday, January 17th at 5PM**. You cannot use slip days on it!
    - See the [Tech Support](https://dsc80.com/tech_support/) page for instructions and watch [this video 🎥](https://www.loom.com/share/0ea254b85b2745e59322b5e5a8692e91?sid=d64a6a06-b2ca-4290-babc-be6666425d09) for tips on how to set up your environment and work on assignments.
    - Please try to set up your computer ASAP, since we have OH on Friday but not over the weekend to help debug your environment.
- Project 1 will be released over the weekend.
- Please fill out the [Welcome Survey](https://docs.google.com/forms/d/e/1FAIpQLSfyspVwdghw5EQShNLyG_L97s0G-X2N8ut8bG6_0K-_WH9DPw/viewform) **ASAP** to help us schedule alternate exams.
- Lecture recordings are available [here](https://podcast.ucsd.edu/watch/wi24/dsc80_a00), and are linked on the course website.

### Agenda

- `numpy` arrays.
- From `babypandas` to `pandas`.
    - Deep dive into DataFrames.
- Accessing subsets of rows and columns in DataFrames.
    - `.loc` and `.iloc`.
    - Querying (i.e. filtering).
- Adding and modifying columns.
- `pandas` and `numpy`.
    
We can't cover every single detail! The [`pandas` documentation](https://pandas.pydata.org/docs/user_guide/index.html#user-guide) will be your friend.

### Throughout lecture, ask questions!

- You're always free to ask questions during lecture, and I'll try to stop for them frequently.
- But, you may not feel like asking your question out loud.
- You can **type your questions anonymously** throughout lecture at the following link:

<div class="alert alert-warning" align=center>
    <h3><a href="https://q.dsc80.com">q.dsc80.com</a></h3>
    <h4>Bookmark it!</h4>
</div>

- I'll check the form responses periodically.
- You'll also use this form to answer questions that I ask you during lecture.

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a href="https://q.dsc80.com">q.dsc80.com)</h3>
</div>
    
How much of the pre-lecture reading did you complete?
    
- A. All of it.
- B. Some of it.
- C. None of it.

## `numpy` arrays

<div class="alert alert-success">
    <h3>Exercise</h3>
    Try and predict the value of <code>grid[-1, 1:].sum()</code> without running the code below.
</div>

In [ ]:
s = (5, 3)
grid = np.ones(s) * 2 * np.arange(1, 16).reshape(s)
# grid[-1, 1:].sum()

### Example: Image processing

As we saw in the pre-lecture reading, `numpy` arrays are homogenous and potentially multi-dimensional.

It turns out that **images** can be represented as 3D `numpy` arrays. The color of each pixel can be described with three numbers under the RGB model – a red value, green value, and blue value. Each of these can vary from 0 to 1.

<center><img src='imgs/three_d_array.png' width=250><small>(<a href="https://e2eml.school/convert_rgb_to_grayscale">image source</a>)</small></center>

In [ ]:
from PIL import Image
img_path = Path('imgs') / 'junior.jpeg'
img = np.asarray(Image.open(img_path)) / 255

In [ ]:
img

In [ ]:
img.shape

In [ ]:
plt.imshow(img)
plt.axis('off');

### Applying a greyscale filter

One way to convert an image to greyscale is to average its red, green, and blue values.

In [ ]:
mean_2d = img.mean(axis=2)
mean_2d

In [ ]:
# This is just a single red channel!
plt.imshow(mean_2d)
plt.axis('off');

We need to _repeat_ `mean_2d` three times along axis 2, to use the same values for the red, green, and blue channels. `np.repeat` will help us here.

In [ ]:
# np.newaxis is an alias for None.
# It helps us introduce an additional axis.
np.arange(5)[:, np.newaxis]

In [ ]:
np.repeat(np.arange(5)[:, np.newaxis], 3, axis=1)

In [ ]:
mean_3d = np.repeat(mean_2d[:, :, np.newaxis], 3, axis=2)

In [ ]:
plt.imshow(mean_3d)
plt.axis('off');

### Applying a sepia filter

Let's sepia-fy Junior!

<center>
<img src="imgs/apple-sepia.png" width=50%>
    <small>
(<a href="https://support.apple.com/guide/motion/sepia-filter-motn169f8c87/mac">Image credits</a>)</small>
</center>

From [here](https://stackoverflow.com/questions/1061093/how-is-a-sepia-tone-created), we can apply this conversion to each pixel.

$$\begin{align*}
R_{\text{sepia}} &= 0.393R + 0.769G + 0.189B \\ G_{\text{sepia}} &= 0.349R + 0.686G + 0.168B \\
B_{\text{sepia}} &= 0.272R + 0.534G + 0.131B\end{align*}$$

In [ ]:
sepia_filter = np.array([
    [0.393, 0.769, 0.189],
    [0.349, 0.686, 0.168],
    [0.272, 0.534, 0.131]
])

In [ ]:
# Multiplies each pixel by the sepia_filter matrix.
# Then, clips each RGB value to be between 0 and 1.
filtered = (img @ sepia_filter.T).clip(0, 1)
filtered

In [ ]:
plt.imshow(filtered)
plt.axis('off');

### Key takeaway: avoid `for`-loops whenever possible!

You can do a lot without `for`-loops, both in `numpy` and in `pandas`.

## From `babypandas` to `pandas` 🐼

### Recap of the pre-lecture reading

- Everything you learned in `babypandas` will translate to `pandas`.

- There are three key data structures in `pandas`:
    - DataFrames: 2 dimensional table.
    - Series: 1 dimensional array-like object, typically representing a column or row.
    - Index: sequence of column or row labels.

- We saw various DataFrame methods: `head`, `tail`, `shape`, `index`, `get`, and `sort_values`.



### Example: Dog Breeds (woof!) 🐶

The dataset we'll work comes from the American Kennel Club. Here's a [cool plot](https://informationisbeautiful.net/visualizations/best-in-show-whats-the-top-data-dog/) made using our dataset.


<center>
    <img src="https://infobeautiful4.s3.amazonaws.com/2014/11/IIB_Best-In-Show_1276x2.png" width=800>
</center>

### Loading in the data

Unfortunately, the original dataset is no longer available, though [this site](https://tmfilho.github.io/akcdata/) has a slightly different version.

Fortunately, we saved the dataset while it was still online.

In [ ]:
all_dogs_path = Path('data') / 'all_dogs.csv'
all_dogs = pd.read_csv(all_dogs_path)
all_dogs

In [ ]:
all_dogs.columns

<div class="alert alert-success">
    <h3>Exercise</h3>
    Find the most popular and least popular dog breeds using the <code>'popularity_all'</code> column in <code>all_dogs</code>.
</div>



In [ ]:
# Your code goes here.

### Too many dogs...

For the rest of the lecture, we'll use a smaller version of the `all_dogs` DataFrame, which will make it easier to demonstrate `pandas` functionality.

In [ ]:
dogs_path = Path('data') / 'dogs43.csv'
dogs = pd.read_csv(dogs_path)
dogs

### Setting the index

Think of each row's index as its **unique identifier** or **name**. Often, we like to set the index of a DataFrame to a unique identifier if we have one available. We can do so with the `set_index` method.

In [ ]:
dogs.set_index('breed')

In [ ]:
# The above cell didn't involve an assignment statement,
# so dogs was unchanged.
dogs

In [ ]:
# By reassigning dogs, our changes will persist.
dogs = dogs.set_index('breed')
dogs

In [ ]:
# There used to be 7 columns, but now there are only 6!
dogs.shape

### 💡 Pro-Tip: Displaying more rows/columns

Sometimes, you just want `pandas` to display a lot of rows and columns. You can use this helper function to do that:

In [ ]:
from IPython.display import display
def display_df(df, rows=pd.options.display.max_rows, cols=pd.options.display.max_columns):
    """Displays n rows and cols from df."""
    with pd.option_context("display.max_rows", rows,
                           "display.max_columns", cols):
        display(df)

In [ ]:
display_df(dogs.sort_values('weight', ascending=False),
           rows=43)

## Selecting columns

### Selecting columns in `babypandas` 👶🐼

- In `babypandas`, you selected columns using the `.get` method.
- `.get` also works in `pandas`, but it is not **idiomatic** – people don't usually use it.

In [ ]:
dogs

In [ ]:
dogs.get('size')

In [ ]:
# This doesn't error, but sometimes we'd like it to.
dogs.get('size oops!')

### Selecting columns with `[]`

* The standard way to select a column in `pandas` is by using the `[]` operator.
* Specifying a column name returns the column as a Series.
* Specifying a list of column names returns a DataFrame.

In [ ]:
dogs

In [ ]:
# Returns a Series.
dogs['kind']

In [ ]:
# Returns a DataFrame.
dogs[['kind', 'size']]

In [ ]:
# 🤔
dogs[['kind']]

In [ ]:
# Breeds are stored in the index, which is not a column!
dogs['breed']

In [ ]:
dogs.index

### Useful Series methods

There are a variety of useful methods that work on Series. You can see the entire list [here](https://pandas.pydata.org/docs/reference/api/pandas.Series.html). Many methods that work on a Series will also work on DataFrames, as we'll soon see.

In [ ]:
dogs

In [ ]:
# What are the unique kinds of dogs?
dogs['kind'].unique()

In [ ]:
# How many unique kinds of dogs are there?
dogs['kind'].nunique()

In [ ]:
# What's the distribution of kinds?
dogs['kind'].value_counts()

In [ ]:
# What's the mean of the 'longevity' column?
dogs['longevity'].mean()

In [ ]:
# Tell me more about the 'weight' column.
dogs['weight'].describe()

In [ ]:
# Sort the 'lifetime_cost' column. Note that here we're using sort_values on a Series, not a DataFrame!
dogs['lifetime_cost'].sort_values()

In [ ]:
# Gives us the index of the largest value, not the largest value itself.
dogs['lifetime_cost'].idxmax()

## Selecting subsets of rows (and columns)

### Use `loc` to _slice_ rows and columns using labels

You saw slicing in DSC 20 and also in the pre-lecture reading.

`loc` works similarly to slicing 2D arrays, but it uses row **labels** and column **labels**, not positions.

In [ ]:
dogs

In [ ]:
# The first argument is the row label.
#        ↓
dogs.loc['Pug', 'longevity']
#                  ↑
# The second argument is the column label.

As an aside, `loc` is _not_ a method – it's an indexer.

In [ ]:
type(dogs.loc)

In [ ]:
type(dogs.sort_values)

### 💡 Pro-Tip: Using Pandas Tutor

If you want, you can install `pandas_tutor` from `pip` in your Terminal (once you've entered your DSC 80 `mamba` environment):

    pip install pandas_tutor

Then, you can load the extension by adding:

    %reload_ext pandas_tutor

At the top of your notebook. After that, you can render visualizations with the `%%pt` cell magic 🪄:

In [ ]:
# Pandas Tutor setup. You'll need to run `pip install pandas_tutor` in your Terminal
# for this cell to work, but you can also ignore the error and continue onward.
%reload_ext pandas_tutor
%set_pandas_tutor_options {"maxDisplayCols": 8, "nohover": True, "projectorMode": True}

In [ ]:
%%pt
dogs.loc['Pug', 'longevity']

### `.loc` is flexible 🧘

You can provide a sequence (list, array, Series) as either argument to `.loc`.

In [ ]:
dogs

In [ ]:
dogs.loc[['Cocker Spaniel', 'Labrador Retriever'], 'size']

In [ ]:
dogs.loc[['Cocker Spaniel', 'Labrador Retriever'], ['kind', 'size', 'height']]

In [ ]:
# Note that the 'weight' column is included!
dogs.loc[['Cocker Spaniel', 'Labrador Retriever'], 'lifetime_cost': 'weight']

In [ ]:
dogs.loc[['Cocker Spaniel', 'Labrador Retriever'], :]

In [ ]:
# Shortcut for the line above.
dogs.loc[['Cocker Spaniel', 'Labrador Retriever']]

### Review: Querying
- As we saw in DSC 10, querying is the act of selecting rows in a DataFrame that satisfy certain condition(s).
- Comparisons with arrays (or Series) result in Boolean arrays (or Series).
- We can use comparisons along with the `loc` operator to **filter** a DataFrame.

In [ ]:
dogs

In [ ]:
dogs.loc[dogs['weight'] < 10]

In [ ]:
dogs.loc[dogs.index.str.contains('Retriever')]

In [ ]:
# Because querying is so common, there's a shortcut:
dogs[dogs.index.str.contains('Retriever')]

In [ ]:
# Empty DataFrame – not an error!
dogs.loc[dogs['kind'] == 'beaver']

Note that because we set the index to `'breed'` earlier, we can select rows based on dog breeds without having to query.

In [ ]:
dogs

In [ ]:
# Series!
dogs.loc['Maltese']

If `'breed'` was instead a column, then we'd need to query to access information about a particular breed.

In [ ]:
dogs_reset = dogs.reset_index()
dogs_reset

In [ ]:
# DataFrame!
dogs_reset[dogs_reset['breed'] == 'Maltese']

### Querying with multiple conditions

Remember, you need parentheses around each condition. Also, you must use the bitwise operators `&` and `|` instead of the standard `and` and `or` keywords. `pandas` makes weird decisions sometimes!

In [ ]:
dogs

In [ ]:
dogs[(dogs['weight'] < 20) & (dogs['kind'] == 'terrier')]

### 💡 Pro-Tip: Using `.query`

`.query` is a convenient way to query, since you don't need parentheses and you can use the `and` and `or` keywords.

In [ ]:
dogs

In [ ]:
dogs.query('weight < 20 and kind == "terrier"')

In [ ]:
dogs.query('kind in ["sporting", "terrier"] and lifetime_cost < 20000')

### Don't forget `iloc`!

- `iloc` stands for "integer location."
- `iloc` is like `loc`, but it selects rows and columns based off of integer positions only, just like with 2D arrays.

In [ ]:
dogs

In [ ]:
dogs.iloc[1:15, :-2]

`iloc` is often most useful when we sort first. For instance, to find the weight of the longest-living dog breed in the dataset:

In [ ]:
dogs.sort_values('longevity', ascending=False)['weight'].iloc[0]

In [ ]:
# Finding the breed itself involves sorting, but not iloc.
dogs.sort_values('longevity', ascending=False).index[0]

### More practice

Consider the DataFrame below.

In [ ]:
jack = pd.DataFrame({1: ['fee', 'fi'], 
                     '1': ['fo', 'fum']})
jack

For each of the following pieces of code, predict what the output will be. Then, uncomment the line of code and see for yourself. We may not be able to cover these all in class; if so, make sure to try them on your own. [Here's a Pandas Tutor link](https://pandastutor.com/vis.html#code=import%20pandas%20as%20pd%0A%0Ajack%20%3D%20pd.DataFrame%28%7B1%3A%20%5B'fee',%20'fi'%5D,%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20'1'%3A%20%5B'fo',%20'fum'%5D%7D%29%0Ajack%5B1%5D&d=2023-10-05&lang=py&v=v1) to visualize these!

In [ ]:
# jack[1]

In [ ]:
# jack[[1]]

In [ ]:
# jack['1']

In [ ]:
# jack[[1, 1]]

In [ ]:
# jack.loc[1]

In [ ]:
# jack.loc[jack[1] == 'fo']

In [ ]:
# jack[1, ['1', 1]]

In [ ]:
# jack.loc[1,1]

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a href="https://q.dsc80.com">q.dsc80.com)</h3>
</div>
    
What questions do you have?

## Adding and modifying columns

### Adding and modifying columns, using a copy

- To add a new column to a DataFrame, use the `assign` method.
    - To change the values in a column, add a new column with the same name as the existing column.
- Like most `pandas` methods, `assign` returns a new DataFrame.
    - **Pro** ✅: This doesn't inadvertently change any existing variables.
    - **Con** ❌: It is not very space efficient, as it creates a new copy each time it is called.

In [ ]:
dogs.assign(cost_per_year=dogs['lifetime_cost'] / dogs['longevity'])

In [ ]:
dogs

### 💡 Pro-Tip: Method chaining

Chain methods together instead of writing long, hard-to-read lines.

In [ ]:
# Finds the rows corresponding to the five cheapest to own breeds on a per-year basis.
(dogs
 .assign(cost_per_year=dogs['lifetime_cost'] / dogs['longevity'])
 .sort_values('cost_per_year')
 .iloc[:5]
)

### 💡 Pro-Tip: `assign` for column names with special characters

You can also use `assign` when the desired column name has spaces (and other special characters) by unpacking a dictionary:

In [ ]:
dogs.assign(**{'cost per year 💵': dogs['lifetime_cost'] / dogs['longevity']})

### Adding and modifying columns, in-place

* You can assign a new column to a DataFrame **in-place** using `[]`.
    - This works like dictionary assignment.
    - This **modifies** the underlying DataFrame, unlike `assign`, which returns a new DataFrame.
* This is the more "common" way of adding/modifying columns. 
    - ⚠️ Warning: Exercise caution when using this approach, since this approach changes the values of existing variables.

In [ ]:
# By default, .copy() returns a deep copy of the object it is called on,
# meaning that if you change the copy the original remains unmodified.
dogs_copy = dogs.copy()
dogs_copy.head(2)

In [ ]:
dogs_copy['cost_per_year'] = dogs_copy['lifetime_cost'] / dogs_copy['longevity']
dogs_copy

Note that we never reassigned `dogs_copy` in the cell above – that is, we never wrote `dogs_copy = ...` – though it was still modified.

### Mutability

DataFrames, like lists, arrays, and dictionaries, are **mutable**. As you learned in DSC 20, this means that they can be modified after being created. (For instance, the list `.append` method mutates in-place.)

Not only does this explain the behavior on the previous slide, but it also explains the following:

In [ ]:
dogs_copy

In [ ]:
def cost_in_thousands():
    dogs_copy['lifetime_cost'] = dogs_copy['lifetime_cost'] / 1000

In [ ]:
# What happens when we run this twice?
cost_in_thousands()

In [ ]:
dogs_copy

### ⚠️ Avoid mutation when possible

Note that `dogs_copy` was modified, even though we didn't reassign it! These unintended consequences can **influence the behavior of test cases on labs and projects**, among other things! 

To avoid this, it's a good idea to avoid mutation when possible. If you must use mutation, include `df = df.copy()` as the first line in functions that take DataFrames as input.

Also, some methods let you use the `inplace=True` argument to mutate the original. **Don't use this argument, since future `pandas` releases plan to remove it.**

## `pandas` and `numpy`

<center><img src='imgs/python-stack.png' width=60%></center>

### `pandas` is built upon `numpy`!

- A Series in `pandas` is a `numpy` array with an index.
- A DataFrame is like a dictionary of columns, each of which is a `numpy` array.
- Many operations in `pandas` are fast because they use `numpy`'s implementations, which are written in fast languages like C.
- If you need access the array underlying a DataFrame or Series, use the `to_numpy` method.

In [ ]:
dogs['lifetime_cost']

In [ ]:
dogs['lifetime_cost'].to_numpy()

### `pandas` data types

- Each Series (column) has a `numpy` data type, which refers to the type of the values stored within. Access it using the `dtypes` attribute.
- A column's data type determines which operations can be applied to it.
- `pandas` tries to guess the correct data types for a given DataFrame, and is often wrong.
    - This can lead to incorrect calculations and poor memory/time performance.
- As a result, you will often need to explicitly convert between data types.

In [ ]:
dogs

In [ ]:
dogs.dtypes

### `pandas` data types

Notice that Python `str` types are `object` types in `numpy` and `pandas`.

|Pandas dtype|Python type|NumPy type|SQL type|Usage|
|---|---|---|---|---|
|int64|int|int_, int8,...,int64, uint8,...,uint64|INT, BIGINT| Integer numbers|
|float64|float|float_, float16, float32, float64|FLOAT| Floating point numbers|
|bool|bool|bool_|BOOL|True/False values|
|datetime64 or Timestamp|datetime.datetime|datetime64|DATETIME|Date and time values|
|timedelta64 or Timedelta|datetime.timedelta|timedelta64|NA|Differences between two datetimes|
|category|NA|NA|ENUM|Finite list of text values|
|object|str|string, unicode|NA|Text|
|object|NA|object|NA|Mixed types|

[This article](https://www.dataquest.io/blog/pandas-big-data/) details how `pandas` stores different data types under the hood.

[This article](https://mortada.net/can-integer-operations-overflow-in-python.html#Can-integers-overflow-in-python?) explains how `numpy`/`pandas` `int64` operations differ from vanilla `int` operations.

### Type conversion

You can change the data type of a Series using the `.astype` Series method.

For example, we can change the data type of the `'lifetime_cost'` column in `dogs` to be `uint32`:

In [ ]:
dogs

In [ ]:
# Gives the types as well as the space taken up by the DataFrame.
dogs.info()

In [ ]:
dogs['lifetime_cost'] = dogs['lifetime_cost'].astype('uint32')

Now, the DataFrame takes up less space! This may be insignificant in our DataFrame, but makes a difference when working with larger datasets.

In [ ]:
dogs.info()

### 💡 Pro-Tip: Setting `dtype`s in `read_csv`

Usually, we prefer to set the correct dtypes in `read_csv`, since it can help `pandas` load in files more quickly:

In [ ]:
dogs_path

In [ ]:
dogs = pd.read_csv(dogs_path, dtype={'lifetime_cost': 'uint32'})
dogs

In [ ]:
dogs.dtypes

### Axes

- The rows and columns of a DataFrame are both stored as Series.
- The **axis** specifies the direction of a **slice** of a DataFrame.

<center><img src='imgs/axis.png' width=30%></center>

- Axis 0 refers to the index (rows).
- Axis 1 refers to the columns.
- **These are the same axes definitions that 2D `numpy` arrays have, as you saw in the pre-lecture reading!**

### DataFrame methods with `axis`

- Many Series methods work on DataFrames.
- In such cases, the DataFrame method usually applies the Series method to every row or column.
- Many of these methods accept an `axis` argument; the default is usually `axis=0`.

In [ ]:
dogs

In [ ]:
# Max element in each column.
dogs.max()

In [ ]:
# Max element in each row – a little nonsensical, since there are different types in each row.
dogs.max(axis=1)

In [ ]:
# The number of unique values in each column.
dogs.nunique()

In [ ]:
# describe doesn't accept an axis argument; it works on every numeric column in the DataFrame it is called on.
dogs.describe()

<div class="alert alert-success">
<h3>Exercise</h3>
Pick a dog breed that you personally like or know the name of. Then:
<ul>
    <li>Try to find a few other dog breeds that are similar in weight to yours in <code>all_dogs</code>.</li>
    <li>Which similar breeds have the lowest and highest <code>'lifetime_cost'</code>? <code>'intelligence_rank'</code>?</li>
    <li>Are there any similar breeds that you haven't heard of before?</li>
</ul>
<br>
    For fun, look up these dog breeds on the <a href="https://www.akc.org/">AKC website</a> to see what they look like!
</div>

<!-- As a bonus, look up these dog breeds on the [AKC website](https://www.akc.org/) to see how they look! -->

In [ ]:
# Your code goes here.

## Summary, next time

### Summary

- `pandas` is **the** library for tabular data manipulation in Python.
- There are three key data structures in `pandas`: DataFrame, Series, and Index.
- Refer to the lecture notebook and the [`pandas` documentation](https://pandas.pydata.org/docs/user_guide/index.html#user-guide) for tips.
- `pandas` relies heavily on `numpy`. An understanding of how data types work in both will allow you to write more efficient and bug-free code.
- Series and DataFrames share many methods (refer to the [`pandas` documentation](https://pandas.pydata.org/docs/user_guide/index.html#user-guide) for more details).
- Most `pandas` methods return copies of Series/DataFrames. Be careful when using techniques that modify values in-place.
- Next time: `groupby` and data granularity.

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a href="https://q.dsc80.com">q.dsc80.com)</h3>
</div>
    
What are your thoughts on the usefulness of the pre-lecture reading? Was it too long? Not as detailed as you were expecting? Released too late?